In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from llm_wrappers.llm_config import OpenAIConfig
from llm_wrappers.utils.chat_interface import chat_ui

from customer_bot import CustomerServiceBot, EndChatInterrupt
from customer_issue import Customer, Order, CustomerIssue

/workspaces/llm-wrapper/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connecting to the database

We will just load a couple of pandas dataframes as our database. Conceptually, this could be a large database with millions of rows.

In [2]:
database = {
    'order_table' : pd.read_csv(
        'data/database/order_table.csv',
        index_col='order_id'),
    'product_table' : pd.read_csv(
        'data/database/product_table.csv',
        index_col='product_id'),
}

database['order_table'].head(n=10)

,order_status,refund_available
order_id,,
10045,Delivered,False
10046,Order Recieved,True
10047,In Transit,True
10048,Delivered,True
10049,In Transit - Delayed,True
10050,Return Initiated,True
10051,Return Complete,False


In [3]:
database['product_table'].head(n=10)

,product_type,product_name,product_price
product_id,,,
51033,Toy,MyFluffy Soft Panda,12.99
51034,TV,"Fefenex 32"" HD TV",130.00
51035,TV,"Tamtung LED TV Ultra 55"" 4K",439.99
51036,Toy,SuperBoys Soldier | toy with lights and sound,8.49
51037,TV,"Jempis Hatak HiQuality 31.5"" TV",119.99
51038,Garden Equipment,Reds Garden Glove,5.99


## Lading the customer service agent

Our customer service agent is a GPT-3.5/4 model, which will be run by making API calls to OpenAI.

In [4]:
load_dotenv()
model_name = 'gpt-3.5-turbo'
api_key = os.getenv('OPENAI_API_KEY')
config = OpenAIConfig(model_name, api_key)

Since we are not doing any fine-tuning, we need to provide instructions to the model in its context window, in the form of a system prompt.

In [5]:
with open('data/sys_prompt.txt', 'r') as f:
    sys_prompt = f.read()

print(sys_prompt)

You are a helpful Customer Support AI Assistant called Darry
the AI Bot, or Darry for short. Your job is to provide support for customers
of the US-based e-commerce website Bamazon Shopping.

You will first receive a message from the system with a JSON formatted list of 
name-value pairs containing some relevant information about the customer, which
you can reference during the conversation.

Then, you will start chatting with the customer. You should first greet the
customer, mention that you are an AI customer support agent, and then ask them
how you can provide support to them. You should chat with the customer in a
brief and professional manner with a natural and polite tone.

During the chat, you can make function calls to request additional information.
If you need some information that is not provided by one of the available
functions, you can call the `sys_request` function to make a query in natural
language that will be processed by a human operator.

Finally, after resolving

We also need to supply the model with information about all the functions it can call, which will be loaded from a JSON file.

In [6]:
with open('data/functions.json', 'r') as f:
    functions = json.load(f)['functions']

# Example of a function
print(json.dumps(functions[0], indent=4))

{
    "name": "is_shipping_available",
    "description": "Whether items can be shipped to city `city_name` in state `state_code`. Returns \"True\" or \"False\"",
    "parameters": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
                "description": "The city, e.g. San Francisco"
            },
            "state_code": {
                "type": "string",
                "description": "The two-character state code, e.g. CA"
            }
        },
        "required": [
            "city_name",
            "state_code"
        ]
    }
}


Finally, we can initialize the agent with all the information we have loaded.

In [7]:
customer_agent = CustomerServiceBot(config, 
    functions = functions,
    database = database)

## Creating a customer issue

We will now create a dummy customer issue to demonstrate how the agent works.

In [8]:
customer = Customer('Sara B', 'Individual', 'Miami, FL', 2)
order = Order(10050, 51034, 'Credit Card')
customer_issue = CustomerIssue(customer, order=order)

## Talking to the agent

We will now talk to the agent, and see how it responds to our queries.

In [9]:
context, greeting_msg = customer_agent.new_chat(sys_prompt, customer_issue)
print(f'Assistant: {greeting_msg}\n')

try:
    context = chat_ui(customer_agent, None, context=context)
except EndChatInterrupt as e:
    print(f'Chat ended: {e}')

Assistant: Hello Sara B! My name is Darry and I'm an AI customer support agent. How can I assist you today?


Assistant: I apologize for the inconvenience, Sara B. It seems that the status of your order with ID 10050 is "Return Initiated". This means that the order is in the process of being returned.

To get more information about the status and location of your package, I suggest contacting our customer support team. They will be able to provide you with the most accurate and up-to-date information. Would you like me to transfer you to a human support expert who can assist you further?


Assistant: Yes, Sara B, it appears that you are eligible for a refund for your order with ID 10050. I apologize for any inconvenience caused. To proceed with the refund, please contact our customer support team. They will assist you further and initiate the refund process.

If you need any assistance in contacting our customer support team or have any further questions, please let me know.

Chat ende